In [31]:
import pandas as pd
import numpy as np
import pathlib 
import os
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm import tqdm
import warnings
import pandas as pd
import argparse

from dataset_paths import *

warnings.filterwarnings('ignore')

output_version = "20241229"
outdir = "/media/hieunguyen/HNSD_mini/outdir"
dataset_name = "LOD"
mode = "all"
PROJECT = "combine_ctcandi_ichorcna"

input_cancer_class = "Liver"

path_to_main_output = os.path.join(outdir, PROJECT, output_version, dataset_name)
path_to_07_output = os.path.join(path_to_main_output, f"07_output_{mode}", input_cancer_class)
path_to_08_output = os.path.join(path_to_main_output, f"08_output_{mode}", input_cancer_class)
path_to_09_output = os.path.join(path_to_main_output, f"09_output_{mode}", input_cancer_class)
os.system(f"mkdir -p {path_to_09_output}")



In [33]:
metadata

,Sample,Actual tumor_fraction_ichorCNA,Simulated TF,spike-in,LABEL,LABCODE,Detected,Decision,RUNTM,RUNGW,SampleID,spike_in_ratio,Label,spike_in_label
30,LODLIV2R1,0.18580,0.15035,50,Liver cancer,LIVER3,Yes,Cancer,R3694,R3694,LODLIV2R1,0.500,Liver,Liver
32,LIVER3R9,0.29280,0.2928,100,Liver cancer,LIVER3,Yes,Cancer,R3685,R3685,LIVER3R9,1.000,Liver,Liver
36,LODLIV7R1,0.03843,0.0015035,0.5,Liver cancer,LIVER3,No,Healthy,R3694,R3694,LODLIV7R1,0.005,Liver,Liver
38,LODLIV7,0.03805,0.001612,0.5,Liver cancer,LIVER3,No,Healthy,R3694,R3694,LODLIV7,0.005,Liver,Liver
42,LODLIV3R2,0.11100,0.02775,25,Liver cancer,LIVER3,Yes,Cancer,R3694,R3694,LODLIV3R2,0.250,Liver,Liver
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,CONTROL44R9,0.03381,Healthy-control,HC,Colorectal cancer,NaN,No,NaN,R3715,R3715,CONTROL44R9,0.000,Control,Control
423,LODCONTROL2,0.02951,Healthy-control,HC,Colorectal cancer,NaN,No,NaN,R3683,R3683,LODCONTROL2,0.000,Control,Control
424,LODCONTROL3,0.02966,Healthy-control,HC,Colorectal cancer,NaN,No,NaN,R3703,R3703,LODCONTROL3,0.000,Control,Control
425,LODCONTROL4R1,0.04060,Healthy-control,HC,Colorectal cancer,NaN,No,NaN,R3715,R3715,LODCONTROL4R1,0.000,Control,Control
